aspect_keywords = ["사용감/착용감", "효과/성능/기능", "품질/디자인/구성", "편의성/활용성", "가격", "사이즈/무게/개수", "제조/유통/서비스"]

# json 파일 만들기

## 폴더 전체에 적용

In [1]:
import pandas as pd
import re
from collections import Counter
import json
import os
from glob import glob

# 불용어 리스트 파일에서 불러오기
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀','것','달바','미스트','스프레이','1으로','1이라','1에','너무','느낌','잘','그냥','많이','있어서','같아요','스프레이가','미스트가','미스트는','약간의',
              '생각보다','있으며','살짝','같아서','느낌이','저는','진짜','엄청','합니다', '더','안','느낌은','효과는','조금','분들은','크림미스트라서',
              '크림미스트라','크림','크림이','미스트라','미스트라서','그런지','못','이거','완전','아벤느','다','수']

# 입력 및 출력 폴더 경로 설정
input_folder = r"\Users\kyn03\OneDrive\바탕 화면\project_file\이탈\합치기"
output_folder = r"\Users\kyn03\OneDrive\바탕 화면\project_file\이탈\워드클라우드_json"
os.makedirs(output_folder, exist_ok=True)

# 여러 속성 키워드 정의
aspect_keywords = ["사용감/착용감", "효과/성능/기능", "품질/디자인/구성", "편의성/활용성", "가격", "사이즈/무게/개수", "제조/유통/서비스"]

def preprocess_text(text_list):
    cleaned_text = [re.sub(r"[^가-힣0-9\+\s]", "", text) for text in text_list]
    cleaned_text = [text for text in cleaned_text if text.strip()]
    return cleaned_text

def save_word_frequencies_to_json(word_freq, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(word_freq, f, ensure_ascii=False)

# 모든 엑셀 파일에 대해 반복 처리
for file_path in glob(os.path.join(input_folder, "*.xlsx")):
    original_file_name = os.path.splitext(os.path.basename(file_path))[0]
    

    # 데이터 로드
    data = pd.read_excel(file_path)
    data = data.where(pd.notna(data), None).reset_index()
    data['index'] = data['index'].astype(str)

    # 각 속성 키워드에 대해 반복하여 JSON 파일 생성
    for aspect_keyword in aspect_keywords:
        sanitized_keyword = aspect_keyword.replace("/", "_")

        filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 5 == 0]]
        if filtered_data.apply(lambda row: row.astype(str).str.contains(aspect_keyword, na=False).any(), axis=1).any():
            aspect_row_data = filtered_data[filtered_data.apply(lambda row: row.astype(str).str.contains(aspect_keyword, na=False).any(), axis=1)]

            positive_b_words = []
            negative_b_words = []
            positive_i_words = []
            negative_i_words = []

            for idx, row in aspect_row_data.iterrows():
                original_idx = data[data['index'] == row['index']].index[0]
                sentiment_row_index = original_idx - 2
                tokenized_text_index = original_idx + 1

                if tokenized_text_index < len(data):
                    for col in data.columns[1:]:
                        if aspect_keyword in str(data.at[original_idx, col]) and pd.notna(data.at[sentiment_row_index, col]) and pd.notna(data.at[tokenized_text_index, col]):
                            sentiment_value = data.at[sentiment_row_index, col]
                            tokenized_text = str(data.at[tokenized_text_index, col])

                            words = [word for word in tokenized_text.split() if not word.startswith('#') and word not in stopwords]
                            cleaned_words = preprocess_text(words)

                            if cleaned_words:
                                if 'B-긍정' in sentiment_value:
                                    positive_b_words.extend(cleaned_words)
                                elif 'B-부정' in sentiment_value:
                                    negative_b_words.extend(cleaned_words)
                                elif 'I-긍정' in sentiment_value:
                                    positive_i_words.extend(cleaned_words)
                                elif 'I-부정' in sentiment_value:
                                    negative_i_words.extend(cleaned_words)

            positive_b_words_freq = Counter(positive_b_words)
            negative_b_words_freq = Counter(negative_b_words)
            positive_i_words_freq = Counter(positive_i_words)
            negative_i_words_freq = Counter(negative_i_words)

            save_word_frequencies_to_json(positive_b_words_freq, os.path.join(output_folder, f'{original_file_name}_{sanitized_keyword}_positive_b_words.json'))
            save_word_frequencies_to_json(positive_i_words_freq, os.path.join(output_folder, f'{original_file_name}_{sanitized_keyword}_positive_i_words.json'))
            save_word_frequencies_to_json(negative_b_words_freq, os.path.join(output_folder, f'{original_file_name}_{sanitized_keyword}_negative_b_words.json'))
            save_word_frequencies_to_json(negative_i_words_freq, os.path.join(output_folder, f'{original_file_name}_{sanitized_keyword}_negative_i_words.json'))

print("모든 파일의 속성별 단어 빈도수가 JSON 파일로 저장되었습니다.")


모든 파일의 속성별 단어 빈도수가 JSON 파일로 저장되었습니다.


In [11]:
import pandas as pd
import re
from collections import Counter
import json
import os

# 불용어 리스트 파일에서 불러오기
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀','것','달바','미스트','스프레이','1으로','1이라','1에','너무','느낌','잘','그냥','많이','있어서','같아요','스프레이가','미스트가','미스트는','약간의',
              '생각보다','있으며','살짝','같아서','느낌이','저는','진짜','엄청','합니다', '더','안','느낌은','효과는','조금','분들은','크림미스트라서',
              '크림미스트라','크림','크림이','미스트라','미스트라서','그런지','못','이거','완전','아벤느','다','수']

# 데이터 로드
file_path = r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_boh.xlsx"
if os.path.splitext(os.path.basename(file_path))[0] == "sorted_merged_boh":
    original_file_name = "바이오힐보"
elif os.path.splitext(os.path.basename(file_path))[0] == "sorted_merged_dalba":
    original_file_name = "달바"
else:
    original_file_name = "아벤느"

data = pd.read_excel(file_path)
data = data.where(pd.notna(data), None)  # 중복된 행은 유지하되, 결측치를 None으로 채움
data = data.reset_index()
data['index'] = data['index'].astype(str)

# 여러 속성 키워드 정의
aspect_keywords = ["사용감/착용감", "효과/성능/기능", "품질/디자인/구성", "편의성/활용성", "가격", "사이즈/무게/개수", "제조/유통/서비스"]

def preprocess_text(text_list):
    # 한글, 숫자, '+' 기호, 공백만 남기고 모든 특수문자 제거
    cleaned_text = [re.sub(r"[^가-힣0-9\+\s]", "", text) for text in text_list]
    # 공백 제거
    cleaned_text = [text for text in cleaned_text if text.strip()]
    return cleaned_text

# JSON 파일로 저장하는 함수
def save_word_frequencies_to_json(word_freq, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(word_freq, f, ensure_ascii=False)

# 각 속성 키워드에 대해 반복하여 JSON 파일 생성
for aspect_keyword in aspect_keywords:
    sanitized_keyword = aspect_keyword.replace("/", "_")  # 파일 이름에 사용할 수 있도록 '/'을 '_'로 대체

    # 'Aspect2 Mapped Labels' 행 필터링 및 특정 키워드 포함 여부 확인
    filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]
    if filtered_data.apply(lambda row: row.astype(str).str.contains(aspect_keyword, na=False).any(), axis=1).any():
        aspect_row_data = filtered_data[filtered_data.apply(lambda row: row.astype(str).str.contains(aspect_keyword, na=False).any(), axis=1)]

        # 긍정/부정 감정 및 토큰화된 텍스트 추출
        positive_b_words = []
        negative_b_words = []
        positive_i_words = []
        negative_i_words = []

        for idx, row in aspect_row_data.iterrows():
            original_idx = data[data['index'] == row['index']].index[0]
            sentiment_row_index = original_idx - 2
            tokenized_text_index = original_idx + 3

            if tokenized_text_index < len(data):
                for col in data.columns[1:]:
                    # None 체크 추가
                    if aspect_keyword in str(data.at[original_idx, col]) and pd.notna(data.at[sentiment_row_index, col]) and pd.notna(data.at[tokenized_text_index, col]):
                        sentiment_value = data.at[sentiment_row_index, col]
                        tokenized_text = str(data.at[tokenized_text_index, col])

                        # 불용어 제거 및 '#'으로 시작하는 단어 제거 후 전처리 적용
                        words = [word for word in tokenized_text.split() if not word.startswith('#') and word not in stopwords]
                        cleaned_words = preprocess_text(words)  # 전처리 함수 호출

                        if cleaned_words:
                            # 감정에 따라 단어 리스트에 추가
                            if 'B-긍정' in sentiment_value:
                                positive_b_words.extend(cleaned_words)
                            elif 'B-부정' in sentiment_value:
                                negative_b_words.extend(cleaned_words)
                            elif 'I-긍정' in sentiment_value:
                                positive_i_words.extend(cleaned_words)
                            elif 'I-부정' in sentiment_value:
                                negative_i_words.extend(cleaned_words)

        # 각 감정별 단어 빈도수 계산
        positive_b_words_freq = Counter(positive_b_words)
        negative_b_words_freq = Counter(negative_b_words)
        positive_i_words_freq = Counter(positive_i_words)
        negative_i_words_freq = Counter(negative_i_words)

        # 각 감정별 JSON 파일로 저장
        save_word_frequencies_to_json(positive_b_words_freq, f'{original_file_name}_{sanitized_keyword}_positive_b_words.json')
        save_word_frequencies_to_json(positive_i_words_freq, f'{original_file_name}_{sanitized_keyword}_positive_i_words.json')
        save_word_frequencies_to_json(negative_b_words_freq, f'{original_file_name}_{sanitized_keyword}_negative_b_words.json')
        save_word_frequencies_to_json(negative_i_words_freq, f'{original_file_name}_{sanitized_keyword}_negative_i_words.json')

print("모든 속성의 단어 빈도수가 JSON 파일로 저장되었습니다.")


모든 속성의 단어 빈도수가 JSON 파일로 저장되었습니다.
